# Acoustic model

## Load

In [1]:
import os

os.environ.putenv("CUDA_VISIBLE_DEVICES", "-1")
config_path = "config/peeter_koik.yaml"
outdir = "./out"

In [2]:
from argparse import ArgumentParser
from utils.config_manager import Config

config_loader = Config(config_path=config_path)
model = config_loader.load_model(None)

2022-02-03 20:14:37.483692: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


2022-02-03 20:14:38.438089: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-02-03 20:14:38.595704: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-03 20:14:38.595732: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kabinet): /proc/driver/nvidia/version does not exist
2022-02-03 20:14:38.596348: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


restored weights from logs/peeter_koik_22/ljspeech/tts_config.aligner_config/weights/latest-311 at step 260000


## Test

In [3]:
text = "Katsetame uut lahendust täna."
phons = model.text_pipeline.phonemizer(text)
tokens = model.text_pipeline.tokenizer(phons)
out = model.predict(tokens, encode=False, phoneme_max_duration=None)
mel = out['mel'].numpy().T

In [4]:
model.build_model_weights()
model.save(os.path.join(outdir, "model"), save_format='tf')
#mel.shape


[WARNING] Unresolved object in checkpoint: (root).step


[WARNING] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.
2022-02-03 20:15:06.528181: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
[WARNING] Found untraced functions such as dropout_layer_call_and_return_conditional_losses, dropout_layer_call_fn, cnn_dropout_layer_call_and_return_conditional_losses, cnn_dropout_layer_call_fn, dense_20_layer_call_and_return_conditional_losses while saving (showing 5 of 960). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: ./out/model/assets


[INFO] Assets written to: ./out/model/assets


# Vocoder

## Load

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm, remove_weight_norm
from vocoding.predictors import HiFiGANPredictor

vocoder = HiFiGANPredictor.from_folder('vocoding/hifigan/en').vocoder_model

for m in vocoder.modules():
    try:
        m_tmp = remove_weight_norm(m)
        m = m_tmp
        #print(m)
    except:
        pass

_ = vocoder.eval()


## Test

In [ ]:
import numpy as np
from data.audio import Audio
audio = Audio(config_loader.config)

In [ ]:
m = mel[np.newaxis, :, :]
t = torch.tensor(m)
a = vocoder(t)
a = a.squeeze()
MAX_WAV_VALUE = 32768.0
a = a * MAX_WAV_VALUE
a = a.detach().numpy().astype('int16')
audio.save_wav(a, os.path.join(outdir, "audio.wav"))

## Save

In [ ]:
script = torch.jit.script(vocoder)
script.save(os.path.join(outdir, "hifigan"))
